In [1]:
import os
import numpy as np
import music21 as m21
import pandas as pd
import json
import matplotlib.pyplot as plt
from scipy import stats
from scipy import spatial
import time
from collections import Counter

np.random.seed(777)
us = m21.environment.UserSettings()

# us['musescoreDirectPNGPath']="/home/sirivasv/Downloads/MuseScore-3.4.2-x86_64.AppImage"
us['musescoreDirectPNGPath']='/home/sirivasv/.local/bin/MuseScore-3.5.2.312125617-x86_64.AppImage'

# Define dataset paths
# MXML_PATH="/media/sirivasv/JASON/Saul/MCC/DATASETS/DATASUBSET/MTC-ANN-2.0.1/mid"
MXML_PATH="/media/sirivasv/DATAL/MCC/DATASUBSET/MTC-ANN-2.0.1/mid"

# METADATA_PATH="/media/sirivasv/JASON/Saul/MCC/DATASETS/DATASUBSET/MTC-ANN-2.0.1/metadata"
METADATA_PATH="/media/sirivasv/DATAL/MCC/DATASUBSET/MTC-ANN-2.0.1/metadata" 

## Data

In [2]:
# Read table of tune family
tune_family_filename = "MTC-ANN-tune-family-labels.csv"
tune_family_df = pd.read_csv(os.path.join(METADATA_PATH, tune_family_filename), header=None)
tune_family_df.head()

,0,1
0,NLB072587_01,Daar_ging_een_heer_1
1,NLB072587_02,Daar_ging_een_heer_1
2,NLB072774_02,Daar_ging_een_heer_1
3,NLB073046_01,Daar_ging_een_heer_1
4,NLB073588_01,Daar_ging_een_heer_1


In [3]:
# Traverse musicxml files and tune family
song_id_x_family = {}
family_x_songs = {}
for root, directories, files in os.walk(MXML_PATH):
    for file in files:
        song_id = file.split(".")[0]
        if (song_id not in song_id_x_family):
            family_name = tune_family_df[tune_family_df[0] == song_id].iloc[0][1]
            song_id_x_family[song_id] = (file, family_name)
            if (family_name not in family_x_songs):
                family_x_songs[family_name] = []
            family_x_songs[family_name].append(song_id)

In [4]:
# Remove the incomplete anotated tunes from the dataframe
reduced_tune_family_df = tune_family_df[tune_family_df[0].isin(list(song_id_x_family.keys()))]
reduced_tune_family_df.head()

,0,1
0,NLB072587_01,Daar_ging_een_heer_1
1,NLB072587_02,Daar_ging_een_heer_1
2,NLB072774_02,Daar_ging_een_heer_1
3,NLB073046_01,Daar_ging_een_heer_1
4,NLB073588_01,Daar_ging_een_heer_1


## Functions

In [5]:
def getSongKey(song):
    key = song.analyze("key")
    return key

In [6]:
def getSongKeyFromMelody_W_Times(melody_w_times_in_k):
    sc_test = m21.stream.Score()
    p0_test = m21.stream.Part()
    p0_test.id = 'part0'
    for pitch_i in melody_w_times_in_k:
        n_i = m21.note.Note(pitch_i[4])
        p0_test.append(n_i)
    sc_test.insert(0, p0_test)
    return getSongKey(sc_test)

In [7]:
# Function to retrieve a list of midi pitch events and its timestamp
def getMelodyDeltaTimes(eventsintrack):
    
    # Initialize array
    DeltaTimes = []
    
    # Initialize cumulative sum
    cum_sum = 0
    
    # Initialize variable to track the time delta
    prev_deltatime = 0
    
    # Traverse the events
    for ev in eventsintrack:
        # If a note starts
        if (ev.isNoteOn()):
            
            # Get the pitch name and save it with the cumulative sum, midi pitch and name
            pitch_in_time = m21.pitch.Pitch(ev.pitch)
            DeltaTimes.append((cum_sum, prev_deltatime, pitch_in_time.midi, pitch_in_time.spanish, pitch_in_time))
            
            # Restart the delta time
            prev_deltatime = 0
        
        # Else if there is a delta time
        elif(str(ev.type) == "DeltaTime"):
            
            # We sum the time
            cum_sum += ev.time
            
            # We sum it to the current delta time
            prev_deltatime += ev.time
    
    # Return the array
    return DeltaTimes

In [8]:
# Read Files 
song_m21_streams = {}

# We traverse the reduced table
for query_row in reduced_tune_family_df.iterrows():
    tune_family_query = query_row[1][1]
    song_id_A = query_row[1][0]
    
    song_stream_A = m21.converter.parseFile(os.path.join(MXML_PATH, song_id_x_family[song_id_A][0]))
    midi_tracks_A = m21.midi.translate.streamToMidiFile(song_stream_A)
    melody_w_times_A = getMelodyDeltaTimes(midi_tracks_A.tracks[0].events)
    
    song_m21_streams[song_id_A] = {
        "song_stream": song_stream_A,
        "midi_tracks": midi_tracks_A,
        "melody_w_times": melody_w_times_A
    }

## Noises

### Type 1. Random Pitch

In [9]:
def get_random_pitch():
    
    new_pitch_class = np.random.randint(0, 12)
    new_pitch_octave = np.random.randint(1, 9)
    
    return m21.pitch.Pitch(octave=new_pitch_octave, pitchClass=new_pitch_class)

In [10]:
# Define apply Transformation type 1: Ruido en notas
def apply_note_noise(melody_w_times_in, percentage=50):
    
    # Track modified notes 
    modified_notes = {}
    
    # Store the length of the melody
    len_melody = len(melody_w_times_in)
    
    # According to the desired percentage of noise we get the number of notes to be modified
    many_notes = int((len_melody * percentage)//100)
    
    for noise_i in range(many_notes):
        
        # Select a random position that we haven't seen yet
        note_to_change = np.random.randint(0, len_melody)
        while (note_to_change in modified_notes):
            note_to_change = np.random.randint(0, len_melody)
        modified_notes[note_to_change] = 1
        
        # Creating a new pitch note
        previous_pitch = melody_w_times_in[note_to_change][3]
        p_new = get_random_pitch()
        while (p_new.spanish == previous_pitch):
            p_new = get_random_pitch()
        
        
        # Replace the data 
        melody_w_times_in[note_to_change] = (
            melody_w_times_in[note_to_change][0],
            melody_w_times_in[note_to_change][1],
            p_new.midi,
            p_new.spanish,
            p_new)
    
    # Return the modified melody
    return melody_w_times_in

### Type 2. Random DeltaTime

In [11]:
def recalculate_timestamps(melody_w_times_in):
    
    # Store the length of the melody
    len_melody = len(melody_w_times_in)
    
    # Define current start time
    current_start_time = 0
    
    # Traverse the melody
    for note_i in range(len_melody):
        current_start_time += melody_w_times_in[note_i][1]
        melody_w_times_in[note_i] = (
            current_start_time,
            melody_w_times_in[note_i][1],
            melody_w_times_in[note_i][2],
            melody_w_times_in[note_i][3],
            melody_w_times_in[note_i][4])
    
    # Return the recalculated melody
    return melody_w_times_in

In [12]:
def get_random_deltatime():
    return np.random.randint(0, 4097)

In [13]:
# Define apply Transformation type 2: Ruido en tiempos
def apply_deltatime_noise(melody_w_times_in, percentage=50):
    
    # Track modified notes 
    modified_notes = {}
    
    # Store the length of the melody
    len_melody = len(melody_w_times_in)
    
    # According to the desired percentage of noise we get the number of notes to be modified
    many_notes = int((len_melody * percentage)//100)
    
    for noise_i in range(many_notes):
        
        # Select a random position that we haven't seen yet
        note_to_change = np.random.randint(0, len_melody)
        while (note_to_change in modified_notes):
            note_to_change = np.random.randint(0, len_melody)
        modified_notes[note_to_change] = 1
        
        # Creating a new deltatime
        previous_deltatime = melody_w_times_in[note_to_change][1]
        deltatime_new = get_random_deltatime()
        while (deltatime_new == previous_deltatime):
            deltatime_new = get_random_deltatime()
        
        # ratio_of_change = np.abs((deltatime_new - previous_deltatime))
        # if previous_deltatime != 0:
        #     ratio_of_change /= previous_deltatime
        # else:
        #     ratio_of_change = -1
        # print("AAA", ratio_of_change)
        
        # Replace the data 
        melody_w_times_in[note_to_change] = (
            melody_w_times_in[note_to_change][0],
            deltatime_new,
            melody_w_times_in[note_to_change][2],
            melody_w_times_in[note_to_change][3],
            melody_w_times_in[note_to_change][4])
        
        # Recalculate timestamps due to the modification in deltatimes
        melody_w_times_in = recalculate_timestamps(melody_w_times_in)
    
    # Return the modified melody
    return melody_w_times_in

### Type 3. Noise in Pitch and Deltatime

In [14]:
# Define apply Transformation type 3: Ruido en tiempos y notas (reemplazo)
def apply_deltatime_and_note_noise(melody_w_times_in, percentage=50):
    
    # Track modified notes 
    modified_notes = {}
    
    # Store the length of the melody
    len_melody = len(melody_w_times_in)
    
    # According to the desired percentage of noise we get the number of notes to be modified
    many_notes = int((len_melody * percentage)//100)
    
    for noise_i in range(many_notes):
        
        # Select a random position that we haven't seen yet
        note_to_change = np.random.randint(0, len_melody)
        while (note_to_change in modified_notes):
            note_to_change = np.random.randint(0, len_melody)
        modified_notes[note_to_change] = 1
        
        # Creating a new deltatime
        previous_deltatime = melody_w_times_in[note_to_change][1]
        deltatime_new = get_random_deltatime()
        while (deltatime_new == previous_deltatime):
            deltatime_new = get_random_deltatime()
        
        # Creating a new pitch note
        previous_pitch = melody_w_times_in[note_to_change][3]
        p_new = get_random_pitch()
        while (p_new.spanish == previous_pitch):
            p_new = get_random_pitch()
            
        # Replace the data 
        melody_w_times_in[note_to_change] = (
            melody_w_times_in[note_to_change][0],
            deltatime_new,
            p_new.midi,
            p_new.spanish,
            p_new)
        
        # Recalculate timestamps due to the modification in deltatimes
        melody_w_times_in = recalculate_timestamps(melody_w_times_in)
    
    # Return the modified melody
    return melody_w_times_in

### Type 4. Removing notes

In [15]:
# Define apply Transformation type 4: Noise by removing events
def apply_removing_noise(melody_w_times_in, percentage=50):
    
    # Store the length of the melody
    len_melody = len(melody_w_times_in)
    
    # According to the desired percentage of noise we get the number of notes to be modified
    many_notes = int((len_melody * percentage)//100)
    
    for noise_i in range(many_notes):
        
        # Select a random position to remove
        note_to_remove = np.random.randint(0, len(melody_w_times_in))
        
        # Remove element
        melody_w_times_in.pop(note_to_remove)
        
        # Recalculate timestamps due to the modification in deltatimes continuity
        melody_w_times_in = recalculate_timestamps(melody_w_times_in)
    
    # Return the modified melody
    return melody_w_times_in

### Type 5. Inserting new notes

In [16]:
# Define apply Transformation type 5: Noise by Inserting events
def apply_inserting_noise(melody_w_times_in, percentage=50):
    
    # Assert only percentages p <= 100 and p > 0
    if percentage >= 100 or percentage < 0:
        percentage = 99
    
    # Store the length of the melody
    len_melody = len(melody_w_times_in)
    
    # According to the desired percentage of noise we get the number of notes to be modified
    new_len = int(len_melody / (1 - (percentage / 100)))
    many_notes = new_len - len_melody
    
    for noise_i in range(many_notes):
        
        # Create new Event
        # Creating a new deltatime
        deltatime_new = get_random_deltatime()
        
        # Creating a new pitch note
        p_new = get_random_pitch()
            
        # Replace the data 
        new_midi_event = (
            0,
            deltatime_new,
            p_new.midi,
            p_new.spanish,
            p_new)
        
        # Select a random position to insert
        pos_to_insert = np.random.randint(0, len(melody_w_times_in))
        
        # Insert element
        melody_w_times_in.insert(pos_to_insert, new_midi_event)
        
        # Recalculate timestamps due to the modification in deltatimes continuity
        melody_w_times_in = recalculate_timestamps(melody_w_times_in)
    
    # Return the modified melody
    return melody_w_times_in

### Noise Controller

In [17]:
def apply_ith_noise(noise_type, melody_w_times_in, percentage=50):
    
    if (noise_type == 1):
        return apply_note_noise(melody_w_times_in, percentage)
    if (noise_type == 2):
        return apply_deltatime_noise(melody_w_times_in, percentage)
    if (noise_type == 3):
        return apply_deltatime_and_note_noise(melody_w_times_in, percentage)
    if (noise_type == 4):
        return apply_removing_noise(melody_w_times_in, percentage)
    
    return apply_inserting_noise(melody_w_times_in, percentage)

In [18]:
def get_MelodyShapeNgram_NOREST(melody_w_times):
    ngram_list = []
    for m_el in melody_w_times:
        # print(m_el)
        current_element = [m_el[2], m_el[0], max(m_el[1],1),  0]
        # print(current_element)
        ngram_list.append(current_element)
    return ngram_list

## Experiments

In [19]:
# song_id_query = "NLB070033_01"
song_id_query = "NLB072967_01"

query_dict_melody_w_times = {"melody_data":[]}

query_melody_w_times = getMelodyDeltaTimes(song_m21_streams[song_id_query]["midi_tracks"].tracks[0].events)

query_dict_melody_w_times["melody_data"] = get_MelodyShapeNgram_NOREST(query_melody_w_times)
    
with open('./Query_Melody_{0}.json'.format(song_id_query), 'w') as outfile:
    json.dump(query_dict_melody_w_times, outfile)

In [20]:
%%time

# Define noise type array
noise_types = [1, 2, 3, 4, 5]

len_noise_types = len(noise_types)

# 
test_melody_arrays = {}

# Define the percentages of noise
noise_percentages = list(map(int, np.linspace(10, 100, 10)))

# We traverse the noises
for noise_type_i in noise_types:

    for noise_percentage_i in noise_percentages:
        test_keyname_i = "{0}_{1}".format(noise_type_i, noise_percentage_i)
        print(test_keyname_i)
        test_melody_arrays[test_keyname_i] = []
        
        melody_w_times_test = getMelodyDeltaTimes(song_m21_streams[song_id_query]["midi_tracks"].tracks[0].events)

        melody_w_times_test = apply_ith_noise(noise_type_i, melody_w_times_test, noise_percentage_i)
        
        test_melody_arrays[test_keyname_i] = get_MelodyShapeNgram_NOREST(melody_w_times_test)


1_10
1_20
1_30
1_40
1_50
1_60
1_70
1_80
1_90
1_100
2_10
2_20
2_30
2_40
2_50
2_60
2_70
2_80
2_90
2_100
3_10
3_20
3_30
3_40
3_50
3_60
3_70
3_80
3_90
3_100
4_10
4_20
4_30
4_40
4_50
4_60
4_70
4_80
4_90
4_100
5_10
5_20
5_30
5_40
5_50
5_60
5_70
5_80
5_90
5_100
CPU times: user 3.72 s, sys: 99 µs, total: 3.72 s
Wall time: 3.72 s


In [21]:
with open('./Test_Melodies_{0}.json'.format(song_id_query), 'w') as outfile:
    json.dump(test_melody_arrays, outfile)